In [1]:
import numpy as np
import pandas as pd


In [2]:
import csv
import numpy as np

cnt = 1
csv_file_path = 'crop_production.csv'
states = {}
districts = {}
season_to_number = {}
crops = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in  csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        states[row[0].strip()] = 1
        districts[row[1].strip()] = 1
        season_to_number[row[3].strip()] = 1
        crops[row[4].strip()] = 1
        
cnt = 1
for val in states.keys():
    states[val] = cnt
    cnt = cnt+1
cnt = 1
for val in districts.keys():
    districts[val] = cnt
    cnt = cnt+1
cnt = 1

for val in season_to_number.keys():
    season_to_number[val] = cnt
    cnt = cnt+1
cnt = 1
for val in crops.keys():
    crops[val] = cnt
    cnt = cnt+1




In [72]:
import csv
import numpy as np


csv_file_path = 'crop_production.csv'

y = []
data = []
cnt = 0
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        try:
            y.append(float(row[6]))
        except:
            continue
        temp_row = row
        if len(temp_row[1]) ==0 : continue
        if len(temp_row[2]) ==0 : continue
        if len(temp_row[3]) ==0 : continue
        if len(temp_row[4]) ==0 : continue
        if len(temp_row[5]) ==0 : continue
        temp_row[1] = districts[temp_row[1].strip()]
        temp_row[2] = int(temp_row[2])
        temp_row[3] = season_to_number[temp_row[3].strip()]
        temp_row[4] = crops[temp_row[4].strip()]
        temp_row[5] = float(temp_row[5])
        
        temp1 = row[1:2]
        temp2 = row[3:6]
        data.append(temp1+temp2)
       
y_train = np.array(y)

X_train = np.array(data)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)

sz = 0.6  # Split ratio (e.g., 80% training, 20% test)

split_index = int(sz * len(X_train_normalized))

X_train_normalized1 = X_train_normalized[:split_index]
y_train1 = y_train[:split_index]
X_test = X_train_normalized[split_index:]
y_test = y_train[split_index:]
# print(X_train_normalized1[0])
print(X_train_normalized1.shape)
print(y_train1.shape)

(145416, 4)
(145416,)


In [4]:
# n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8 

In [53]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
param_grid_gb = {
              'learning_rate': [1e-07],
              'min_child_weight': [12],
              'max_depth' : [4],
              'n_estimators' : [100],
              'gamma' : [0.00001],
              'subsample' : [0.9],
              'colsample_bytree': [1],
              'reg_alpha' : [1e-8]
                }
model = XGBRegressor()
mse_grid = GridSearchCV(estimator = model, param_grid = param_grid_gb, scoring = 'neg_mean_squared_error', cv = 4, verbose = 1,
                        n_jobs = -1)

mse_grid.fit(X_train_normalized1,y_train1)

print("Best parameter: ", mse_grid.best_params_)

# Procedure: First tune min_child_weight and max_depth
# Tune gamma
# Tune subsample and colsample
# Tune regularization (alpha paramater)
# Tune the estimators as required

# Previous run (27m 35.4s) Best parameter:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
# Best parameter:  {'colsample_bytree': 1e-05, 'gamma': 0, 'learning_rate': 1e-07, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 1e-07, 'subsample': 1}


Fitting 4 folds for each of 4 candidates, totalling 16 fits
Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}


In [165]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV

# param_random_gb = {
#               'learning_rate': np.arange(0.05,0.15, 0.05),
#               # 'n_estimators' : np.arange(200,1000,100)
#             #   'subsample' : np.arange(0.9,1, 0.05),
#             #   "colsample_bytree" : np.arange(0.9,1,0.05)
#               }

# # Regressor Instantiation
# gb = XGBRegressor()

# # Here we specified a total of 25 iterations
# mse_random = RandomizedSearchCV(estimator = gb, param_distributions= param_random_gb, n_iter = 25, 
#                                 scoring = 'neg_mean_squared_error', cv = 4, verbose = 1)

# mse_random.fit(X_train_normalized1,y_train1)

# print("Best parameter: ", mse_random.best_params_)

In [100]:
# Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}


model = XGBRegressor(n_estimators=10000)
model.fit(X_train_normalized1,y_train1)

# print("Best parameter: ", mse_grid.best_params_)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [76]:
def error_prediction(X,y) :
    err = 0
    m,n = X.shape
    print(m)
    for i in range(0,m):
        temp = np.zeros((1,n))
        temp[0] = X[i]
        err += (model.predict(temp)-y[i])**2
    err/=m
    return err

In [101]:
# training_error = 0
from sklearn.metrics import mean_squared_error
print("Training error:")
print(mean_squared_error(y_train1,model.predict(X_train_normalized1)))
print("Test error:")
print(mean_squared_error(y_test,model.predict(X_test)))

# Training error:
# 175811405285739.38
# Test error:
# 184513262020819.66

Training error:
2041076666.4211206
Test error:


ValueError: Found input variables with inconsistent numbers of samples: [96945, 1]

In [102]:
# This will be used for user inputs
# State_Name = states['Andaman and Nicobar Islands']
District_Name = districts['NICOBARS']
# Crop_Year = 2000
Season = season_to_number['Kharif']
Crop = crops['Arecanut']
Area = 102.00
# print(State_Name,District_Name,Crop_Year,Season,Crop,Area)
X_test = np.zeros((1,4))
X_test[0] = np.array([District_Name,Season,Crop,Area])
X_test_normalized = scaler.transform(X_test)

Production = model.predict(X_test_normalized)
print(X_test_normalized)
print(Production)

[[-1.65923103 -0.97610996 -1.21405365 -0.2372404 ]]
[512.9985]
